In [ ]:
!pip3 install scikit-learn==1.0 -U
!pip install -q xlrd
!git clone https://github.com/tjshippey/ESEM2016
!pip install PrettyTable
!pip install category_encoders

from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
import glob
import sys,os
import pathlib
from prettytable import PrettyTable
import imblearn
from imblearn.ensemble import BalancedBaggingClassifier, RUSBoostClassifier, EasyEnsembleClassifier
import numpy as np
import pandas as pd
from sklearn import model_selection
from sklearn.impute import SimpleImputer
from sklearn.ensemble import ExtraTreesClassifier
from imblearn.ensemble import BalancedBaggingClassifier,BalancedRandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as score1
from sklearn.metrics import recall_score, precision_score,roc_auc_score , make_scorer,f1_score
from sklearn.model_selection import cross_val_score ,cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import BaggingClassifier,GradientBoostingClassifier,RandomForestClassifier,AdaBoostClassifier
from numpy import mean


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 23.1 MB 1.3 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
Cloning into 'ESEM2016'...
remote: Enumerating objects: 173, done.
remote: Total 173 (delta 0), reused 0 (delta 0), pack-reused 173
Receiving objects: 100% (173/173), 94.49 MiB | 36.71 MiB/s, done.
Resolving deltas: 100% (119/119), done.
Checking out files: 100% (144/144), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 861 kB/s 


In [ ]:
def check_data(filepath):#check for unary class
  raw_df = pd.read_csv(filepath,sep="\t|,", engine='python')
  check_df = raw_df.copy()
  check_df.pop('Package')
  check_df.pop('Class')
  check_df.pop('Method')
   #check for unary class
  if check_df[check_df.columns[29]].nunique()==2:
    del raw_df
    return 2
  elif check_df[check_df.columns[29]].nunique()==1:
    del raw_df
    return 1

def process_data(filepath):
 raw_df = pd.read_csv(filepath,sep="\t|,",engine='python')
 raw_df =raw_df.fillna(method='backfill') #Replace missing data
 neg, pos = np.bincount(raw_df[raw_df.columns[32]])
 total = neg + pos  
 percentage=100 * pos / total  
#Drop package_name,class_name and method_name columns we do not need them in our classification model
 cleaned_df = raw_df.copy()
 cleaned_df.pop('Package')#Drop ID-Like metrics
 cleaned_df.pop('Class')#Drop ID-Like metrics
 cleaned_df.pop('Method')#Drop ID-Like metrics
 cleaned_df['FLAG'] = pd.factorize(cleaned_df.FLAG)[0] + 1
 cleaned_df =cleaned_df.fillna('backfill')

 for column in cleaned_df.columns:#Normalize data
       if not(cleaned_df.columns[29]  ) :
        cleaned_df[column] = (cleaned_df[column] - cleaned_df[column].min()) / (cleaned_df[column].max() - cleaned_df[column].min())    
 labels=cleaned_df[cleaned_df.columns[29]]
 labels=labels.astype('int')
 features=cleaned_df[cleaned_df.columns[0:28]]
 del cleaned_df

 return features,labels 

#//////////////////////////////////////////////////////////
def Ensemble_model_bagging(features,labels) :
    model= BaggingClassifier().fit(features,labels)
    kfold = model_selection.KFold(n_splits=10)
    scoring = scoring = ['precision', 'recall', 'f1','roc_auc']
    results = cross_validate(estimator=model,
                                          X=features,
                                          y=labels,
                                          cv=kfold,
                                          scoring=scoring)
    return results   
def Ensemble_model_rf(features,labels) :
    classifier =RandomForestClassifier()
    model=RandomForestClassifier().fit(features,labels)
    kfold = model_selection.KFold(n_splits=10)
    scoring = scoring = ['precision', 'recall', 'f1','roc_auc']
    results = cross_validate(estimator=model,
                                          X=features,
                                          y=labels,
                                          cv=kfold,
                                          scoring=scoring)
    return results   
    
def Ensemble_model_Adaboost(features,labels) :
    model=AdaBoostClassifier().fit(features,labels)
    kfold = model_selection.KFold(n_splits=10)
    scoring = scoring = ['precision', 'recall', 'f1','roc_auc']
    results = cross_validate(estimator=model,
                                          X=features,
                                          y=labels,
                                          cv=kfold,
                                          scoring=scoring)
    return results  
    
def Ensemble_model_gradiant(features,labels) :
    model=GradientBoostingClassifier().fit(features,labels)
    kfold = model_selection.KFold(n_splits=10)
    scoring = scoring = ['precision', 'recall', 'f1','roc_auc']
    results = cross_validate(estimator=model,
                                          X=features,
                                          y=labels,
                                          cv=kfold,
                                          scoring=scoring)
    return results  
def Balanced_RF_model(features,labels) :
    model =BalancedRandomForestClassifier(n_estimators=100, random_state=0)
    model.fit(features,labels)
    kfold = model_selection.KFold(n_splits=10)
    scoring = scoring = ['precision', 'recall', 'f1','roc_auc']
    results = cross_validate(estimator=model,
                                          X=features,
                                          y=labels,
                                          cv=kfold,
                                          scoring=scoring)
    return results  
def Balanced_bagging_model(features,labels) :
    model = BalancedBaggingClassifier()
    model.fit(features,labels)
    kfold = model_selection.KFold(n_splits=10)
    scoring = scoring = ['precision', 'recall', 'f1','roc_auc']
    results = cross_validate(estimator=model,
                                          X=features,
                                          y=labels,
                                          cv=kfold,
                                          scoring=scoring)
    return results 
def Balanced_RUSboost_model(features,labels) :
    model = RUSBoostClassifier()
    model.fit(features,labels)
    kfold = model_selection.KFold(n_splits=10)
    scoring = scoring = ['precision', 'recall', 'f1','roc_auc']
    results = cross_validate(estimator=model,
                                          X=features,
                                          y=labels,
                                          cv=kfold,
                                          scoring=scoring)
    return results     
def easy_ensemble(features,labels) :
    model = EasyEnsembleClassifier()
    model.fit(features,labels)
    kfold = model_selection.KFold(n_splits=10)
    scoring = scoring = ['precision', 'recall', 'f1','roc_auc']
    results = cross_validate(estimator=model,
                                          X=features,
                                          y=labels,
                                          cv=kfold,
                                          scoring=scoring)
    return results     
#######################################################################
currentDirectory = pathlib.Path('/content/ESEM2016/MethodLevel')
table =PrettyTable()
table.field_names = ["Dataset","prescision","recall","roc_auc"]
l=[]

for currentFile in currentDirectory.iterdir() : # iterate through directory to get datasets 
   precesion =0.0
   recall    =0.0 
   f1        =0.0
   roc_auc   =0.0   
  
   filename= currentFile.name
   
   if  filename  not in l :  #make sure the dataset has not already retrived
    if check_data(currentFile)==2: # check if the dataset has defective class
       i=0
       bi_dataset=filename
       l.append(bi_dataset)
       print(l)
       features,labels=process_data(currentFile)
     #  results= Ensemble_model_Adaboost(features,labels)
       #results=Balanced_RUSboost_model(features,labels)
       results=Balanced_bagging_model(features,labels)
       #results=Ensemble_model_bagging(features,labels)
       #results=Ensemble_model_rf(features,labels)
      # results=Ensemble_model_gradiant(features,labels)
      # results=Balanced_RF_model(features,labels) 
        # results=easy_ensemble(features,labels)
       for name  in results.keys():
         
            temp=np.round(np.average(results[name]),2)
            if i==5:
             roc_auc=temp
             
            if  i==4:
             f1=temp 
            if i==3:
             recall=temp 
            
            if i==2:
             precesion=temp
           
     #skip fit_time and score_time returned by results   at i=0 and i=1  
            i=i+1  
       
       table.add_row([bi_dataset ,precesion,recall,roc_auc])
     

  


['metricsFlagsDefects_xaware5.csv']
['metricsFlagsDefects_xaware5.csv', 'metricsFlagsDefects_ejdt3.csv']
['metricsFlagsDefects_xaware5.csv', 'metricsFlagsDefects_ejdt3.csv', 'metricsFlagsDefects_jikesrvm3.csv']
['metricsFlagsDefects_xaware5.csv', 'metricsFlagsDefects_ejdt3.csv', 'metricsFlagsDefects_jikesrvm3.csv', 'metricsFlagsDefects_jmol2.csv']
['metricsFlagsDefects_xaware5.csv', 'metricsFlagsDefects_ejdt3.csv', 'metricsFlagsDefects_jikesrvm3.csv', 'metricsFlagsDefects_jmol2.csv', 'metricsFlagsDefects_jmri2.4.csv']
['metricsFlagsDefects_xaware5.csv', 'metricsFlagsDefects_ejdt3.csv', 'metricsFlagsDefects_jikesrvm3.csv', 'metricsFlagsDefects_jmol2.csv', 'metricsFlagsDefects_jmri2.4.csv', 'metricsFlagsDefects_controltier3.1.csv']
['metricsFlagsDefects_xaware5.csv', 'metricsFlagsDefects_ejdt3.csv', 'metricsFlagsDefects_jikesrvm3.csv', 'metricsFlagsDefects_jmol2.csv', 'metricsFlagsDefects_jmri2.4.csv', 'metricsFlagsDefects_controltier3.1.csv', 'metricsFlagsDefects_jmol9.csv']
['metricsFl

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:775: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 103, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 381, in _score
    return self._sign * self._score_func(y, y_pred, **self._kwarg

['metricsFlagsDefects_xaware5.csv', 'metricsFlagsDefects_ejdt3.csv', 'metricsFlagsDefects_jikesrvm3.csv', 'metricsFlagsDefects_jmol2.csv', 'metricsFlagsDefects_jmri2.4.csv', 'metricsFlagsDefects_controltier3.1.csv', 'metricsFlagsDefects_jmol9.csv', 'metricsFlagsDefects_omegat3.6.csv', 'metricsFlagsDefects_drjava2010.csv', 'metricsFlagsDefects_cmusphinx3.7.csv', 'metricsFlagsDefects_unicore1.5.csv']
['metricsFlagsDefects_xaware5.csv', 'metricsFlagsDefects_ejdt3.csv', 'metricsFlagsDefects_jikesrvm3.csv', 'metricsFlagsDefects_jmol2.csv', 'metricsFlagsDefects_jmri2.4.csv', 'metricsFlagsDefects_controltier3.1.csv', 'metricsFlagsDefects_jmol9.csv', 'metricsFlagsDefects_omegat3.6.csv', 'metricsFlagsDefects_drjava2010.csv', 'metricsFlagsDefects_cmusphinx3.7.csv', 'metricsFlagsDefects_unicore1.5.csv', 'metricsFlagsDefects_tango2008.csv']
['metricsFlagsDefects_xaware5.csv', 'metricsFlagsDefects_ejdt3.csv', 'metricsFlagsDefects_jikesrvm3.csv', 'metricsFlagsDefects_jmol2.csv', 'metricsFlagsDefects

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:775: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 103, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 381, in _score
    return self._sign * self._score_func(y, y_pred, **self._kwarg

['metricsFlagsDefects_xaware5.csv', 'metricsFlagsDefects_ejdt3.csv', 'metricsFlagsDefects_jikesrvm3.csv', 'metricsFlagsDefects_jmol2.csv', 'metricsFlagsDefects_jmri2.4.csv', 'metricsFlagsDefects_controltier3.1.csv', 'metricsFlagsDefects_jmol9.csv', 'metricsFlagsDefects_omegat3.6.csv', 'metricsFlagsDefects_drjava2010.csv', 'metricsFlagsDefects_cmusphinx3.7.csv', 'metricsFlagsDefects_unicore1.5.csv', 'metricsFlagsDefects_tango2008.csv', 'metricsFlagsDefects_genoviz6.1.csv', 'metricsFlagsDefects_jump1.5.csv', 'metricsFlagsDefects_unicore1.3.csv', 'metricsFlagsDefects_htmlunit2010.csv']
['metricsFlagsDefects_xaware5.csv', 'metricsFlagsDefects_ejdt3.csv', 'metricsFlagsDefects_jikesrvm3.csv', 'metricsFlagsDefects_jmol2.csv', 'metricsFlagsDefects_jmri2.4.csv', 'metricsFlagsDefects_controltier3.1.csv', 'metricsFlagsDefects_jmol9.csv', 'metricsFlagsDefects_omegat3.6.csv', 'metricsFlagsDefects_drjava2010.csv', 'metricsFlagsDefects_cmusphinx3.7.csv', 'metricsFlagsDefects_unicore1.5.csv', 'metrics

...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:775: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 103, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 381, in _score
    return self._sign * self._score_func(y, y_pred, **self._kwarg

['metricsFlagsDefects_xaware5.csv', 'metricsFlagsDefects_ejdt3.csv', 'metricsFlagsDefects_jikesrvm3.csv', 'metricsFlagsDefects_jmol2.csv', 'metricsFlagsDefects_jmri2.4.csv', 'metricsFlagsDefects_controltier3.1.csv', 'metricsFlagsDefects_jmol9.csv', 'metricsFlagsDefects_omegat3.6.csv', 'metricsFlagsDefects_drjava2010.csv', 'metricsFlagsDefects_cmusphinx3.7.csv', 'metricsFlagsDefects_unicore1.5.csv', 'metricsFlagsDefects_tango2008.csv', 'metricsFlagsDefects_genoviz6.1.csv', 'metricsFlagsDefects_jump1.5.csv', 'metricsFlagsDefects_unicore1.3.csv', 'metricsFlagsDefects_htmlunit2010.csv', 'metricsFlagsDefects_runawfe4.1.csv', 'metricsFlagsDefects_omegat3.5.csv', 'metricsFlagsDefects_jppf4.2.csv', 'metricsFlagsDefects_jump1.7.csv', 'metricsFlagsDefects_cmusphinx3.6.csv', 'metricsFlagsDefects_cdk1.1.csv', 'metricsFlagsDefects_eclemma2.1.csv', 'metricsFlagsDefects_jmol4.csv', 'metricsFlagsDefects_jikesrvm2.csv', 'metricsFlagsDefects_jmri2.csv', 'metricsFlagsDefects_jppf5.csv', 'metricsFlagsDefe

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:775: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 103, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 381, in _score
    return self._sign * self._score_func(y, y_pred, **self._kwarg

['metricsFlagsDefects_xaware5.csv', 'metricsFlagsDefects_ejdt3.csv', 'metricsFlagsDefects_jikesrvm3.csv', 'metricsFlagsDefects_jmol2.csv', 'metricsFlagsDefects_jmri2.4.csv', 'metricsFlagsDefects_controltier3.1.csv', 'metricsFlagsDefects_jmol9.csv', 'metricsFlagsDefects_omegat3.6.csv', 'metricsFlagsDefects_drjava2010.csv', 'metricsFlagsDefects_cmusphinx3.7.csv', 'metricsFlagsDefects_unicore1.5.csv', 'metricsFlagsDefects_tango2008.csv', 'metricsFlagsDefects_genoviz6.1.csv', 'metricsFlagsDefects_jump1.5.csv', 'metricsFlagsDefects_unicore1.3.csv', 'metricsFlagsDefects_htmlunit2010.csv', 'metricsFlagsDefects_runawfe4.1.csv', 'metricsFlagsDefects_omegat3.5.csv', 'metricsFlagsDefects_jppf4.2.csv', 'metricsFlagsDefects_jump1.7.csv', 'metricsFlagsDefects_cmusphinx3.6.csv', 'metricsFlagsDefects_cdk1.1.csv', 'metricsFlagsDefects_eclemma2.1.csv', 'metricsFlagsDefects_jmol4.csv', 'metricsFlagsDefects_jikesrvm2.csv', 'metricsFlagsDefects_jmri2.csv', 'metricsFlagsDefects_jppf5.csv', 'metricsFlagsDefe

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:775: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 103, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 381, in _score
    return self._sign * self._score_func(y, y_pred, **self._kwarg

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
table.sortby="Dataset"     #sort table alphapiticaly
data=table

with open('model_results.csv', 'w', newline='') as f_output:
    f_output.write(data.get_csv_string())#please open te table with excel and use average function to get the average of each coloumn as python 
    # numpy average  function returns nan if there is a (nan) value 

# New Section